In [1]:
import pandas as pd
import numpy as np
import re

# Chapter 8

## 8.1 Hierarchical Indexing

hierarchical indexing enables you to have multiple index levels on an axis. example: create a Series with a list of lists as the index:

In [2]:
data = pd.Series(np.random.randn(9),
                index= [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                       [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    1.185779
   2    1.436521
   3   -0.706687
b  1    1.041711
   3   -1.454098
c  1    0.683225
   2   -1.016601
d  2   -0.207848
   3    2.225874
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
#partial indexing possible with a hiercrchically indexed object. makes it easy to select subsets of data:
data['b']

1    1.041711
3   -1.454098
dtype: float64

In [6]:
data['b':'c']

b  1    1.041711
   3   -1.454098
c  1    0.683225
   2   -1.016601
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    1.041711
   3   -1.454098
d  2   -0.207848
   3    2.225874
dtype: float64

In [8]:
#selection possible from an "inner" level
data.loc[:, 2]

a    1.436521
c   -1.016601
d   -0.207848
dtype: float64

hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. example: could rearrange the data into a DF using its _unstack_ method:


In [9]:
data.unstack()

,1,2,3
a,1.185779,1.436521,-0.706687
b,1.041711,NaN,-1.454098
c,0.683225,-1.016601,NaN
d,NaN,-0.207848,2.225874


In [10]:
#inverse operation of unstack is stack
data.unstack().stack()

a  1    1.185779
   2    1.436521
   3   -0.706687
b  1    1.041711
   3   -1.454098
c  1    0.683225
   2   -1.016601
d  2   -0.207848
   3    2.225874
dtype: float64

**with a DF, either axis can have a hierarchical index:**

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'],[1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']])

In [12]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ['key1', 'key2']

In [14]:
frame.columns.names = ['state', 'color']

In [15]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
#with partial column indexing you can similarly select groups of columns:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

a MultiIndex can be created by itself and then reused; the columns in the preceding DF with level names could be created like this:

In [17]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                          ['Green', 'Red', 'Green']],
                         names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

sometimes may need to rearrange the order of the levels, or sort the data by the values in a specific level.

the _swaplevel_ takes two level numbers or names and returns a **new object** with the levels interchanged (but the data is otherwise unaltered)

In [18]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

_sort_index_ sorts the data using only the values in a single level. when swapping levels, it's not uncommon to also use _sort_index_ so that the result is lexicographically sorted by the indicated level:

In [19]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

You can specify the level you want to aggregate on by a particular axis on DF and Series.

In [22]:
frame.sum(level='key2')

/var/folders/rw/1rnjc7b90vd_mr3753j04_s40000gp/T/ipykernel_44708/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(level='color', axis=1)

/var/folders/rw/1rnjc7b90vd_mr3753j04_s40000gp/T/ipykernel_44708/4133796543.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's Columns

you may want to use one or more columns from a DF as the rows index; or you might want to move the row index into the DF's columns. example:


In [24]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})

In [25]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [26]:
#DF's set_index func will create a new DF using one or more of its columns as the index:
frame2 = frame.set_index(['c', 'd'])

In [27]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

**by default the columns are removed from the DF, though you can leave them in:**

In [28]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

_reset_index_ does the opposite of _set_index_ and moves the hierarchical index levels into columns:


In [29]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

can combine data in multiple ways:  
  
  - pandas.merge connects rows in DFs based one one or more keys. similar to SQL database join operations     
  - pandas.concat concatenates or "stacks" together objects along an axis.  
  - the combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another

### Database-Style DataFrame Joins

_merge_ or _join_ operations combine datasets by linking rows using one or more _keys_

In [30]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})

In [31]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                   'data2': range(3)})

In [32]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [33]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [34]:
#example of many to one join: the data in df1 has multiple rows labeled a and b, 
#where df2 only has one row for each value in the key column

pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


did not specify which column to join on. merge defaults to using the overlapping column names as the keys. good practice to specify explicitly:

In [35]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


if column names are different in each object, can specify them separately:


In [36]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})

In [37]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                   'data2': range(3)})

In [38]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


notice 'c' and 'd' values and associated data are missing. by default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables. other possible options are 'left', 'right', and 'outer'. the outer join takes the union of the keys, combining the effect of applying both left and right joins:


In [39]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [40]:
#many to many merge example:

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

In [41]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                   'data2': range(5)})

In [42]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [43]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [44]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


many to many joins form the cartesian product of the rows. the join method only affects the distinct key values appearing in the result:

In [45]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [46]:
#to merge with multiple keys, pass a list of column names:

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})

In [47]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'rval': [4, 5, 6, 7]})

In [48]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


when joining columns on columns, the indexes on the passed Df objects are discarded

one more issues is overlapping column names. you can address manually (renaming axis labels), or merge has a suffixes option for specifying strings to append to overlapping names in the left and right DF objects:

In [49]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [51]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

in some cases, the merge key(s) in a DF will be found in its index. you can pass left_index=True or right_index=True to indicate that the index should be used as the merge key:

In [52]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})

In [53]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [54]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [55]:
right1

,group_val
a,3.5
b,7.0


In [56]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


since the default merge is to intersect the join keys, you can instead form the union of them with an outer join:

In [57]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


with hierarchically indexed data, things are more complicated as joining on index is implicitly a multiple-key merge:

In [58]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                     'key2': [2000, 2001, 2002, 2001, 2002],
                     'data': np.arange(5.)})

In [60]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
            index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                  [2001, 2000, 2000, 2000, 2001, 2002]],
            columns=['event1', 'event2'])

In [61]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [62]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

in this case, you have to indicate multiple columns to merge on as a list(note the handling of duplicate index values with how='outer'

In [63]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [64]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
        right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [65]:
#using index of both sides of merge

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=['a', 'c', 'e'],
                    columns=['Ohio', 'Nevada'])

In [66]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                     columns=['Missouri', 'Alabama'])

In [67]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [68]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [69]:
pd.merge(left2, right2, how='outer',  left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [70]:
#DF has join instance for merging by index. for previous example could have written:

left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [71]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [73]:
#for simple index on index merges, can pass a list of DFs to join

another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon'])


In [74]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [76]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [77]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

concatenation, binding, or stacking are all another kind of datacombination.

In [79]:
arr = np.arange(12).reshape(3, 4)

In [80]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [81]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [82]:
#three Series with no index overlap:

s1 = pd.Series([0, 1], index=['a', 'b'])

In [84]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [85]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [86]:
#calling concat with these objects in a list glues together the values and indexes:

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

by default concat works along axis = 0, producing another Series. If you pass axis=1, the result will instead be a DF(axis=1 is the column):

In [87]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


in this case there is no overlap on the otheraxis, which as you can see is the osrted union(the 'outer' join) of the indexes. You can instead intersect them by passing join='inner'

In [88]:
s4 = pd.concat([s1, s3])

In [89]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [90]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [91]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [ ]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) #depreciated join_axes won't work

In [93]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [94]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [95]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [98]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) #keys become DF column headers

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [99]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                  columns=['one', 'two'])

In [100]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                  columns=['three', 'four'])

In [101]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [102]:
df2

,three,four
a,5,6
c,7,8


In [104]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [105]:
pd.concat({'level1': df1, 'level2': df2}, axis=1) #dict keys will be used for the keys option

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [107]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [108]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [109]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [110]:
df1

,a,b,c,d
0,-0.333787,0.870018,2.253248,0.016822
1,-1.024004,0.496724,-1.733597,-0.117090
2,0.602468,-1.337628,-0.448883,0.493430


In [111]:
df2

,b,d,a
0,-0.240645,0.330070,-1.133476
1,-0.444840,0.950373,0.095044


In [112]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.333787,0.870018,2.253248,0.016822
1,-1.024004,0.496724,-1.733597,-0.117090
2,0.602468,-1.337628,-0.448883,0.493430
3,-1.133476,-0.240645,NaN,0.330070
4,0.095044,-0.444840,NaN,0.950373


### Combining Data with Overlap

In [113]:
#two datasets whose indexes overlap in full or part

a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [114]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
               index=['f', 'e', 'd', 'c', 'b', 'a'])


In [115]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [116]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [117]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

Series has a _combine_first_ method which performs the equivalent of this operation along with panda's usual data alignment logic:

In [118]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

with DFs _combine_first_ does the same thing column by column, so you can think of it as "patching" missing data in the calling object with data from the object you pass:

In [119]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

In [120]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})


In [121]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [122]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [123]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

stack - rotates or pivots from the columns in the data to the rows.    
unstack - pivots from the rows into the columns

In [124]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))

In [125]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [126]:
result = data.stack()

In [127]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [128]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


by default the innermost level is unstacked/stacked. you can unstack a different level by passing a level number or name

In [129]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [130]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


unstacking might introduce missing data if all of the values in the level aren't found in each subgroup:

In [131]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [132]:
s2 = pd.Series([4, 5, 6], index=['c','d', 'e'])

In [133]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [134]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [135]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


stacking filters out missing data by default, so the operationis more easily invertible

In [137]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [138]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [139]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [140]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

In [141]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [142]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [143]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format

In [144]:
data = pd.read_csv('examples/macrodata.csv')

In [145]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [146]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

In [147]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [148]:
data = data.reindex(columns=columns)

In [149]:
data.index = periods.to_timestamp('D', 'end')

In [151]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [152]:
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [153]:
pivoted = ldata.pivot('date', 'item', 'value')

In [154]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


the first two values passed are the columns to be used respectively as the row and column index, then finally an optional value column to fill the DF. 

In [155]:
# have 2 value columns that you wanted to reshape simultaneously:

ldata['value2'] = np.random.randn(len(ldata))

In [156]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.910914
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.224815
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.292367
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.434119
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.670204
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.899399
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.068762
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.998028
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.676200
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.393568


In [157]:
#by omitting the last argument, you obtain a DF with hierarchical columns:

pivoted = ldata.pivot('date', 'item')

In [158]:
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.224815 -0.910914   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.670204 -0.434119   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.998028 -0.068762   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.830096 -0.393568   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.359541 -0.389109   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.292367  
1959-06-30 23:59:59.999999999 -0.899399  
1959-09-30 23:59:59.999999999 -0.676200  
1959-12-31 23:59:59.999999999  0.592751  
1960-03-31 23:59:59.999999999  0.117635

In [159]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


**pivot is dquivalent to creating a hierarchical index using set_index followed by a call to unstack:**

In [160]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [161]:
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.224815 -0.910914   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.670204 -0.434119   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.998028 -0.068762   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.830096 -0.393568   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.359541 -0.389109   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.625350 -0.265464   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.455896 -0.448841   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.292367  
1959-06-30 23:59:59.999999999 -0.899399  
1959-09-30 23:59:59.999999999 -0.676200  
1959-12-31 23:59:59.999999999  0.592751  
1960-03-31 23:59:59.999999999  0.117635  
1960-06-30 23:59:59.999999999  1.175621  
1960-09-30 23:59:59.999999999  0.363664

### Pivoting "Wide" to "Long" Format

inverse to pivot is pandas.melt. rather than transforming one column into many in a new DF, it merges multiple columns into one, producing a DF that is longer than the input.

In [162]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})

In [163]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


the'key' column may be a group indicator, and the other columns are data values. when using pandas.metl, you must indicate which column (if any) are group indicators. 

In [164]:
melted = pd.melt(df, ['key'])

In [165]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [166]:
#using pivot to reshape back to original layout:

reshaped = melted.pivot('key', 'variable', 'value')

In [167]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [168]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [169]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [170]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [171]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## 8.4 Conclusion